# Test benchmark

## Setup

In [1]:
import sys
import subprocess
from antidote import __version__, is_compiled
print(f"""
== Python ==
{sys.version}

== Antidote =
{__version__} {'(cython)' if is_compiled() else ''}
""")


== Python ==
3.9.1 (default, Dec  7 2020, 22:33:43) 
[GCC 9.3.0]

== Antidote =
0.12.1 (cython)



In [9]:
cat /proc/cpuinfo | grep 'model name' | head -n 1

model name	: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz


### Imitating a really big project

Creating 4000 different dependencies, all singletons, in all the different ways that actually impact test utilities.

In [2]:
from antidote import factory, Service, implementation, Factory

n = 1000

services = [
    type(f"Service{i}", (Service,), {})
    for i in range(n)
]

def make_factory(Output):
    @factory
    def f() -> Output:
        return Output()
    
    return f

factories = dict()
for i in range(n):
    Output = type(f"Ouput{i}", (object,), {})
    factories[Output] = make_factory(Output)

def make_implementation(Interface, Impl):
    @implementation(Interface)
    def f():
        return Impl
    
    return f

def make_factory_method(Output):
    def f(self) -> Output:
        return Output()
    
    return f

factories = dict()
for i in range(n, 2 * n):
    Output = type(f"Ouput{i}", (object,), {})
    factories[Output] = type(f"Factory{i}", (Factory,), {"__call__": make_factory_method(Output)})

    
implementations = dict()
for i in range(n):
    Interface = type(f"Interface{i}", (object,), {})
    Impl = type(f"Impl{i}", (Interface, Service), {})
    implementations[Interface] = make_implementation(Interface, Impl)

# Instantiating all dependencies
from antidote import world

for service in services:
    world.get(service)

for output, factory in factories.items():
    world.get(output @ factory)
    
for interface, impl in implementations.items():
    world.get(interface @ impl)

## Benchmarks

In [3]:
def bench_empty():
    with world.test.empty():
        pass
    
%timeit bench_empty()

8.59 µs ± 546 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
def bench_new():
    with world.test.new():
        pass
    
%timeit bench_new()

29.4 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
def bench_clone():
    with world.test.clone():
        pass
    
%timeit bench_clone()

70.6 µs ± 3.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
def bench_clone_keeping_singletons():
    with world.test.clone(keep_singletons=True):
        pass
    
%timeit bench_clone_keeping_singletons()

123 µs ± 2.98 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
x = object()
with world.test.clone():
    %timeit world.test.override.singleton(services[0], x)

1.81 µs ± 20 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
x = object()

def static():
    return x

with world.test.clone():
    %timeit world.test.override.factory(services[0])(static)

2.66 µs ± 33.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
